In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [0]:
import json

In [0]:
from distribution_spatial_footprint.isochrones import isochronesManager
from distribution_spatial_footprint.geographic_handling import compute_polygon_envelope
from distribution_spatial_footprint.utils import from_minutes_to_seconds, from_seconds_to_minutes, prettify_isochrone_label
from dku_utils import get_current_project_and_variables

In [0]:
project, variables = get_current_project_and_variables()
global_variables = variables["standard"]

In [0]:
api_key = global_variables["api_key"]
isochrones_api_to_use = global_variables["isochrones_api_to_use"]
isochrones_api_parameters = global_variables[isochrones_api_to_use]
isochrones_range_min = global_variables["isochrones_range_min"]
points_of_interest_tags = global_variables["points_of_interest_tags"]

In [0]:
# Read recipe inputs
INPUT_DATASET_NAME = "locations_prepared" # locations_prepared_sampled / locations_prepared
locations_prepared = dataiku.Dataset(INPUT_DATASET_NAME)
locations_prepared_df = locations_prepared.get_dataframe()

In [0]:
locations_identifiers = list(locations_prepared_df["location_identifier"])

# Isochrones computation

In [0]:
isochrones_manager = isochronesManager(isochrones_api_to_use, api_key, isochrones_api_parameters, isochrones_range_min)

In [0]:
latitudes = list(locations_prepared_df["latitude"])
longitudes = list(locations_prepared_df["longitude"])
isochrones = isochrones_manager.compute_isochrones_features(latitudes, longitudes)

In [0]:
isochrones_manager.extract_isochrone_attributes(isochrones[5][0])

In [0]:
isochrones_attributes_to_extract = isochrones_manager.isochrones_attributes
n_attributes_to_extract = len(isochrones_attributes_to_extract)
for isochrone_type in isochrones.keys():
    isochrone_type_attributes = {attribute:[] for attribute in isochrones_attributes_to_extract}
    isochrones_focus = isochrones[isochrone_type]

    for isochrone_index, zip_data in enumerate(zip(isochrones_focus, locations_identifiers)):
        isochrone = zip_data[0]
        isochrone_location = zip_data[1]
        isochrone_polygon = isochrone["geometry"]["coordinates"][0]
        isochrone_attributes = isochrones_manager.extract_isochrone_attributes(isochrone)

        for attribute in isochrones_attributes_to_extract:
            isochrone_type_attributes[attribute].append(isochrone_attributes[attribute])

        isochrone_envelope, isochrone_envelope_str = compute_polygon_envelope(isochrone_polygon)

    locations_prepared_df["isochrone_{}_min".format(isochrone_type)] = isochrones[isochrone_type]
    locations_prepared_df["isochrone_{}_min".format(isochrone_type)] =\
    locations_prepared_df["isochrone_{}_min".format(isochrone_type)].apply(lambda x: json.dumps(x))

    for attribute in isochrones_attributes_to_extract:
        locations_prepared_df["isochrone_{}_min_{}".format(isochrone_type, attribute)] = \
        isochrone_type_attributes[attribute]

In [0]:
# Webapp | Location id :
locations_prepared_df["location_id"] = range(len(locations_prepared_df))

In [0]:
isochrones_types = ["isochrone_{}_min".format(isochrone_amplitude) for isochrone_amplitude in isochrones_range_min]

In [0]:
locations_isochrones_denormalized_df = pd.DataFrame(columns=["isochrone_id", "location_id", "isochrone_amplitude",
                                                             "isochrone_type","isochrone_label", "isochrone_data"])

In [0]:
for row in locations_prepared_df[["location_id"]+isochrones_types].iterrows():
    row_data = row[1]
    location_id = row_data["location_id"]

    for isochrone_amplitude, isochrones_type in zip(isochrones_range_min, isochrones_types):
        isochrone_id = "{}_{}".format(location_id, isochrone_amplitude)
        locations_isochrones_denormalized_df =\
        locations_isochrones_denormalized_df.append({"isochrone_id":isochrone_id,
                                                     "location_id":location_id,
                                                     "isochrone_amplitude":isochrone_amplitude,
                                                     "isochrone_type":isochrones_type,
                                                     "isochrone_label":prettify_isochrone_label(isochrones_type),
                                                     "isochrone_data":row_data[isochrones_type]
                                                    },
                                                    ignore_index=True)

In [0]:
locations_isochrones = dataiku.Dataset("locations_isochrones")
locations_isochrones.write_with_schema(locations_prepared_df)

In [0]:
locations_isochrones_denormalized = dataiku.Dataset("locations_isochrones_denormalized")
locations_isochrones_denormalized.write_with_schema(locations_isochrones_denormalized_df)